In [0]:
--###################################################################################
-- DECLARANDO E SETANDO AS VARIÁVEIS, NÃO ESQUEÇA DE DEFINIR O PARAMETRO "INICIAIS" #
--###################################################################################
DECLARE catalogo string;
SET var catalogo = concat('workshop_databricks_', :iniciais);

--##################################################
-- CRIAÇÃO DO CATALOGO E SCHEMA                    #
--##################################################
CREATE  CATALOG IF NOT EXISTS IDENTIFIER(catalogo) ;
USE CATALOG IDENTIFIER(catalogo);
CREATE SCHEMA IF NOT EXISTS agents;

USE CATALOG IDENTIFIER(catalogo);
USE SCHEMA agents;

--##################################################
-- CRIAÇÃO DO VOLUME                               #
--##################################################

CREATE VOLUME IF NOT EXISTS agents.vol_agent;

--##################################################
-- CRIAÇÃO DAS TABELAS                             #
--##################################################
CREATE TABLE IF NOT EXISTS pacientes (
  nome STRING,
  cpf STRING,
  endereco STRING,
  plano STRING);

CREATE TABLE IF NOT EXISTS medicos (
  nome_medico STRING,
  endereco STRING,
  cidade STRING,
  estado STRING,
  pais STRING,
  cep STRING,
  latitude STRING,
  longitude STRING,
  especialidade STRING,
  planos_aceitos ARRAY<STRING>,
  bairro STRING);
  
--###################################################################################################
-- PREPARANDO A TABELA PACIENTES USANDO COPY INTO COM FORCE=TRUE, PARA VIABILIZAR O REPROCESSAMENTO #
--###################################################################################################
TRUNCATE TABLE pacientes;
COPY INTO pacientes
FROM '/Volumes/workshop_databricks_jsf/agents/vol_agent/pacientes.csv'
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true')
COPY_OPTIONS (
 'force' = 'true'
);

--############################################################
-- PREPARANDO DADOS DA TABELA MEDICOS, UTILIZANDO READ_FILES #
--############################################################
TRUNCATE TABLE medicos;
INSERT INTO medicos
SELECT nome_medico,endereco,cidade,estado,pais,cep,latitude,longitude, especialidade, from_json(planos_aceitos,'ARRAY<STRING>') planos_aceitos, bairro FROM read_files(
  '/Volumes/workshop_databricks_jsf/agents/vol_agent/medicos.csv',
  format => 'csv',
  quote => '"',
  escape => '"',
  header => true
);

--######################################################
-- VERIFICANDO SE AMBAS TABELAS POSSUEM 1000 REGISTROS #
--######################################################
SELECT 'A tabela tabela médicos tem: '|| string(count(1))|| ' registros' resultado from medicos
UNION ALL
SELECT 'A tabela pacientes tem: '|| string(count(1) || ' registros') from pacientes